<a href="https://colab.research.google.com/github/Aryu-Tamura/GoogleColab_RAG_test/blob/main/RAG%E3%82%92%E7%94%A8%E3%81%84%E3%81%9FLLM%E3%81%AE%E5%87%BA%E5%8A%9B%E6%A4%9C%E8%A8%BC%E3%83%AC%E3%83%9D%E3%83%BC%E3%83%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**RAGを用いたLLMの出力検証レポート**

###**（1）やったこと・動かしたものの概要**

RAGを実装し、出力の精度について考察した。具体的には、"RAGを実装した場合"、"追加データなしで使用したLLMモデルに依存する場合”、"RAGで分割する前のテキストデータをそのままプロンプトとして入力した場合"、のそれぞれで出力結果の比較を行った。



###**（2）具体的な手続き**

Google Colab上で以下のプログラムを実行した。コーディングはGeminiを活用して行った。

きっかけ：
松尾研LLMコミュニティ【Paper&Hacks #43】の井伊篤彦氏による「RAG vs ロングコンテクストアプリ・ハンズオン」に参加したこと。



In [ ]:
# ライブラリのインストール
!pip install openai
!pip install langchain
!pip install chromadb
!pip install tiktoken
!pip install -U langchain-community
!pip install langchain-openai

# ライブラリのインポート
import os
from google.colab import files
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from openai import OpenAI

In [ ]:
# OpenAI API ログイン
from google.colab import userdata
import openai

key = userdata.get('OpenAI_key')
os.environ["OPENAI_API_KEY"] = key
openai.api_key = key
client = OpenAI()

print("OpenAI APIキーが設定されました。")

Wikipediaから取得した「ONE PIECE」に関する約1万7000文字のテキストデータアップロードしている。データの整理などは行っていない。

In [ ]:
# ファイルアップロード
uploaded = files.upload()
file_name = next(iter(uploaded))
raw_text = uploaded[file_name].decode('utf-8')
print(f"\nアップロードされたファイル '{file_name}' の内容 (先頭300文字):\n{raw_text[:300]}...\n")

In [ ]:
# 質問リスト
questions = [
    "アニメ『ONE PIECE』の放送開始当初の放送曜日は何曜日でしたか？ ",
    "アニメ『ONE PIECE』で、2023年12月に制作が発表された、原作を「東の海編」から再アニメ化する新アニメシリーズのタイトルは何ですか？ ",
    "アニメ『ONE PIECE』で、麦わらの一味の声優が他のキャラクターの声を兼任する際に使われる別名義の冒頭の名称は何ですか？",
    "アニメ『ONE PIECE』が放送1000話に到達したのは西暦何年ですか？ "
]

# RAGありで質問応答を行う関数
def rag_based_qa(question, retriever, llm):
    qa = RetrievalQA.from_llm(llm=llm, retriever=retriever, return_source_documents=True)
    result = qa({"query": question})
    print(f"\n質問: {question}")
    print("回答 (RAGあり):", result["result"])
    print("参照元ドキュメント:")
    for doc in result["source_documents"]:
        print(f"  {doc.page_content[:100]}...")
    return result["result"]

# RAGなしで質問応答を行う関数
def no_rag_qa(question, llm):
    response = client.chat.completions.create(
        model=llm.model_name,
        messages=[
            {"role": "user", "content": question}
        ]
    )
    print(f"\n質問: {question}")
    print("回答 (RAGなし):", response.choices[0].message.content)
    return response.choices[0].message.content

# RAGなしでテキスト全体をプロンプトに含めて質問応答を行う関数
def no_rag_with_context_qa(question, text, llm):
    prompt_with_context = f"以下のテキストに基づいて質問に答えてください。\n\n{text}\n\n質問: {question}"
    response = client.chat.completions.create(
        model=llm.model_name,
        messages=[
            {"role": "user", "content": prompt_with_context}
        ]
    )
    print(f"\n質問: {question}")
    print("回答 (RAGなし/全文):", response.choices[0].message.content)
    return response.choices[0].message.content

# テキスト分割
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
documents = text_splitter.create_documents([raw_text])

# Embeddingモデルの初期化
embeddings = OpenAIEmbeddings()

# ベクトルストアの構築
db = Chroma.from_documents(documents, embeddings)
retriever = db.as_retriever(search_kwargs={"k": 3})

# LLMの初期化 (gpt-4o-mini を使用)
llm = ChatOpenAI(model_name="gpt-4o-mini")

print("--- RAG あり ---")
rag_results = {}
for question in questions:
    rag_results[question] = rag_based_qa(question, retriever, llm)

print("\n--- RAG なし ---")
no_rag_results = {}
for question in questions:
    no_rag_results[question] = no_rag_qa(question, llm)

print("\n--- RAG なし (テキスト全体をプロンプトに含めて与える) ---")
no_rag_context_results = {}
for question in questions:
    no_rag_context_results[question] = no_rag_with_context_qa(question, raw_text, llm)

print("\n--- 結果比較 ---")
for question in questions:
    print(f"\n質問: {question}")
    print(f"  回答 (RAGあり): {rag_results[question]}")
    print(f"  回答 (RAGなし): {no_rag_results[question]}")
    print(f"  回答 (RAGなし/全文): {no_rag_context_results[question]}")

Pandasで結果をわかりやすく可視化する。

In [ ]:
import pandas as pd
from IPython.display import display, HTML

# 結果データ（再掲）
results_data = [
    {"質問": "アニメ『ONE PIECE』の放送開始当初の放送曜日は何曜日でしたか？", "正解": "水曜日", "結果": "○/○/○"},
    {"質問": "アニメ『ONE PIECE』で、2023年12月に制作が発表された、原作を「東の海編」から再アニメ化する新アニメシリーズのタイトルは何ですか？", "正解": "『THE ONE PIECE』", "結果": "○/H/○"},
    {"質問": "アニメ『ONE PIECE』で、麦わらの一味の声優が他のキャラクターの声を兼任する際に使われる別名義の冒頭の名称は何ですか？", "正解": "粗忽屋（そこつや）", "結果": "×/H/○"},
    {"質問": "アニメ『ONE PIECE』が放送1000話に到達したのは西暦何年ですか？", "正解": "2021年", "結果": "○/○/○"}
]

# pandas DataFrameの作成
df = pd.DataFrame(results_data)
df.columns = ["質問", "正解", "結果 [A]/[B]/[C]"]

# DataFrameをHTMLとして表示（より綺麗で見やすい）
display(HTML(df.to_html(index=False)))

# 注意書き
print("\n--- 注意 ---")
print("結果の表記:")
print("○: 正解")
print("×: 回答なし")
print("H: ハルシネーション（事実に基づかない内容を含む回答）")
print("[A]: アップロードされたデータから検索（RAG）")
print("[B]: gpt-4o-miniにデータを与えずに直接質問")
print("[C]: アップロードされたデータ全体を入力")

###**（3）わかったこと・わからなかったこと**


**わかったこと**


RAGを実装したLLMの出力結果を分析したところ、質問3において「情報が提供されていない」という回答が得られた。正解となる情報自体はシステムに提供していたにも関わらず、このような出力になった点は、一見すると期待外れかもしれない。しかし、特筆すべきは、この状況下でLLMがハルシネーション（事実に基づかない情報生成）を起こさなかったという点である。これは、エンタープライズ環境、特にコールセンターのような正確性が求められるチャットボット開発においては非常に重要な特性と言える。誤った情報を出力するリスクを最小限に抑え、確実な情報のみを提供する姿勢は、RAGの大きな利点であると考えらる。一方、追加データなしでLLMに直接質問した場合、4問中2問でハルシネーションが発生しており、RAGが情報の信頼性を高める上で有効であることがわかった。


興味深い結果として、RAGとしてではなく、アップロードしたデータの全文をプロンプトとしてLLMに与えた場合、正答率は100%を達成した。近年のLLMはコンテキストウィンドウ（一度に入力可能なプロンプトの量）が飛躍的に拡大しており、このアプローチも現実的な選択肢となりつつあると考えらる。しかしながら、入力するデータが極めて大規模になるケースでは、この手法は入力トークン数の上限を超える可能性があり、適用が難しい場面も想定される。


そこで、RAGと全文入力を組み合わせたハイブリッドなアプローチの可能性を感じた。例えば、行政の補助金・助成金案内のチャットボットを想定する。多岐にわたる助成金情報を全てプロンプトに含めるのは非現実的である。しかし、ユーザーが自身の属性やニーズをチェックマークなどで選択し、関連性の高い情報群を絞り込むことができれば、その絞り込まれた比較的小規模なデータをプロンプトの全文としてLLMに提供することが可能である。この手法であれば、大規模なデータ全体を扱うことなく、ユーザーが必要とする情報を高い精度でLLMから引き出せるのではないかと考えた。


**わからなかったこと**


今回の検証では、アップロードしたデータの整理や前処理をほとんど行わなかった。文献調査などを通して、より効果的なデータのクリーニングや構造化の手法を学び、それをRAGに適用することで、LLMの出力精度が向上する可能性は大いにあると考えられる。今後は、関連する論文を読み解き、データ整理の実装に挑戦していきたい。


また、今回の検証で使用したLLMはgpt-4o-miniのみだった。OpenAI以外の主要なLLM、例えばGoogleのGeminiやAnthropicのClaudeといったモデルでも同様の実験を行い、それぞれの特性やRAGとの相性を比較することで、より深い知見が得られると考えられる。異なるモデルでの挙動を把握することは、実際のアプリケーション開発において最適なモデルを選択する上で不可欠なステップだと考えられる。